# Kinetic Mechanism Pathway Analyzer

This analyzer is designed to provide a better user experience and improved analyzing capacity (e.g., large mechanisms) compared with Chemkin mechanism analyzer. This analyzer takes reaction ROP file (e.g. Chemkin ckcsv file), builds a reaction network, helps answer

- if the mechanism has certain pathway (from start species to end species)

- what is the biggest flux from/towards a certain species

When using this script, please pay attention to the sections marked with **[user input]** which require user to specify certain input parameters.

In [11]:
from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy.chemkin import loadChemkinFile
import os

## [user input] Prepare files

Please add 

- annotated chemkin files (mechanism and species dictionary) and 

- chemkin simulation ckcsv file 

to a folder with name stored in `mech` variable in the next cell.

In [12]:
mech = 'minimal'

In [13]:
mechPath = os.path.join('./', 'data', 'pathway_analysis', mech)
chemkinPath= os.path.join(mechPath, 'chem.inp')
dictionaryPath = os.path.join(mechPath, 'species_dictionary.txt')
ckcsvPath= mechPath + '/CKSoln.ckcsv'

model = CoreEdgeReactionModel()
model.core.species, model.core.reactions = loadChemkinFile(chemkinPath,dictionaryPath)

In [14]:
# generate paris for reactions that don't have flux pairs
for rxn in model.core.reactions:
    if not rxn.pairs: rxn.generatePairs()

## Create reaction network

In [15]:
import networkx as nx
import matplotlib.pyplot as plt
from rmgpy.tools.extractInfoFromckcsv import getConcentrationDictFromCKCSV, getROPFromCKCSV, getFluxGraphEdgesDict, showTheMostDominantSpecies
from rmgpy.chemkin import getSpeciesIdentifier
from IPython.display import display
import numpy as np
from rmgpy.rmg.pdep import PDepReaction
%matplotlib inline

### step1: prepare a dict for graph edges

In [16]:
firstColDict, spc_mf_dict = getConcentrationDictFromCKCSV(ckcsvPath)
first_col_dict, spc_total_rop_dict, spc_rop_dict = getROPFromCKCSV(ckcsvPath)

graph_edges_dict = getFluxGraphEdgesDict(spc_rop_dict, model.core.reactions)

graph_edges_dict_simple = {}
for pair in graph_edges_dict:
    node1 = getSpeciesIdentifier(pair[0])
    node2 = getSpeciesIdentifier(pair[1])
    graph_edges_dict_simple[(node1, node2)] = graph_edges_dict[pair]

### [user input] step2: pick a time to analyze pathways

Please specify 

- the time point you want to investigate

- the chemkin name of your initial species/reactant

- the reaction temperature in K

- the reaction pressure in Pa if the mechanism includes pressure-dependent reactions

- showIntegratedFlux: True if users want to use time-integrated fluxes (integrated up to the investigated time point) for analysis. Flase if users want to use instantaneous fluxes at the investigated time point for analysis

In [43]:
# User Input
time_investigated = 0.000001 # hour
reactant_name = 'ethane(1)'
T = 1350 # K
P = 100000 # Pa
showIntegratedFlux = True # If True, all flux will be the total flux integrated up to the investigated time point. 
# If False, all flux will be the instantaneous flux at the investigated time point


timepoint_index = (np.abs(firstColDict['Time_(sec)']-time_investigated*3600)).argmin()
print "Investigated time point is {0} secs".format(firstColDict['Time_(sec)'][timepoint_index])
print "At this moment, the reactant's mole fraction is {}.".format(spc_mf_dict[reactant_name][timepoint_index])

# Create moleFraction_dict at the investigated time point
moleFraction_dict = {}
for i, v in spc_mf_dict.items():
    moleFraction_dict.update({i:v[timepoint_index]})

# Create species_identifier_didct for the core species    
species_identifier_dict = {}
for i, s in enumerate(model.core.species):
    species_identifier_dict.update({getSpeciesIdentifier(model.core.species[i]): s})

# find the total consumption of the reactant in mol/cm^3 (positive value means consumption)
total_reactant_consumption = 0 # 
for timepoint in range(timepoint_index-1):
    #integrate the flux using a trapezoidal rule
    total_reactant_consumption += 0.5 * (spc_total_rop_dict[reactant_name][1][timepoint+1] + spc_total_rop_dict[reactant_name][1][timepoint]) * (firstColDict['Time_(sec)'][timepoint+1] - firstColDict['Time_(sec)'][timepoint])
total_reactant_consumption = total_reactant_consumption * -1.0

if showIntegratedFlux:
    print "At this moment, the reactant's total consumption is {0:.3E} mol/vol.".format(total_reactant_consumption)

# create DiGraph for this moment
G = nx.DiGraph()
for pair in graph_edges_dict:
    node1 = getSpeciesIdentifier(pair[0])
    node2 = getSpeciesIdentifier(pair[1])
    e_rawdata = graph_edges_dict[pair]
    total_flux = 0
    for rxn in e_rawdata:
        if showIntegratedFlux:
            #integrate the flux using a trapezoidal rule
            for timepoint in range(timepoint_index-1):
                total_flux += 0.5 * (e_rawdata[rxn][timepoint+1] + e_rawdata[rxn][timepoint]) * (firstColDict['Time_(sec)'][timepoint+1] - firstColDict['Time_(sec)'][timepoint])
        else:
            total_flux += e_rawdata[rxn][timepoint_index]
    if total_flux >= 0:
        G.add_edge(node2, node1, {"total_flux":total_flux}) # in G, positive means production of node1
    else: 
        G.add_edge(node1, node2, {"total_flux":-total_flux}) # in G, negative means consumption of node1      

Investigated time point is 0.003731082 secs
At this moment, the reactant's mole fraction is 0.2634996.
At this moment, the reactant's total consumption is 5.068E-06 mol/vol.


## [user input] Functionality 1: find most dominant species at the investigated time point

Pleae specify

- dominant_list: If you want to visualize the first top 10 dominant species, specify the list as [0, 9]. If you want to visualize from the 5th most dominant species to the 10th most dominant species, specify the list as [4, 9].

- displayMW: If you want to visualize the species' molecular weight as well, set displayMW as 'True'. If not, set it as 'False'

In [44]:
dominant_list = [0, 9] 
displayMW = True

In [45]:
showTheMostDominantSpecies(dominant_list, moleFraction_dict, species_identifier_dict, displayMW)

****The most dominant species****


H2(12): 0.3409702 (mole fraction)
mw = 2

****The 2nd most dominant species****


C2H4(8): 0.2822238 (mole fraction)
mw = 28

****The 3rd most dominant species****


ethane(1): 0.2634996 (mole fraction)
mw = 30

****The 4th most dominant species****


C(6): 0.06701013 (mole fraction)
mw = 16

****The 5th most dominant species****


...

C2H5(3): 0.0001190951 (mole fraction)
mw = 29

****The 7th most dominant species****


CH3(2): 6.491413e-05 (mole fraction)
mw = 15

****The 8th most dominant species****


C2H3(13): 1.159101e-05 (mole fraction)
mw = 27

****The 9th most dominant species****


H(4): 1.066908e-05 (mole fraction)
mw = 1

****The 10th most dominant species****


C3H7(14): 1.040717e-07 (mole fraction)
mw = 43



## [user input] Functionality 2: find paths between two species

Pleae specify

- source species

- target species

In [46]:
src_species = 'ethane(1)'
tgt_species = 'C#C(25)'

## output1: sort out all the paths and most significant one

**If IntegratedFlux is chosen, output also displays the relative flux, which is the percent flux relative to the reactant flux integrated up to the investigated time point. 

In [47]:
paths = list(nx.all_simple_paths(G, source=src_species, target=tgt_species, cutoff=5))

In [48]:
path_fluxes = []
if showIntegratedFlux:
    print '========================================================================================================'
    print 'Path index \t Integrated Flux \t Rel. Flux (%) \t\t Path steps'
else:
    print '================================================'
    print 'Path index \t Flux \t\t Path steps'
print ''

for i, path in enumerate(paths):
    
    path_steps = len(path) - 1
    fluxes = [G[path[step]][path[step+1]]['total_flux'] for step in range(path_steps) ]
    path_fluxes.append(min(fluxes))
    if showIntegratedFlux:
        print 'Path #{0}: \t {1:.3E} \t\t {2:.2f} \t\t {3}'.format(i, min(fluxes), min(fluxes)/total_reactant_consumption*100, path)
    else:
        print 'Path #{0}: \t {1:.3E} \t {2}'.format(i, min(fluxes), path)
sorted_path_fluxes = sorted(path_fluxes)

print ''
print '======================================================='
print '\t Most Significant Path '
print '======================================================='
if showIntegratedFlux:
    print 'Path index \t Integrated Flux \t Rel. Flux (%)'
    print 'Path #{0}: \t {1:.3E} \t\t {2:.2f}'.format(path_fluxes.index(sorted_path_fluxes[-1]), sorted_path_fluxes[-1], sorted_path_fluxes[-1]/total_reactant_consumption*100)
else:
    print 'Path index \t Flux'
    print 'Path #{0}: \t {1:.3E} '.format(path_fluxes.index(sorted_path_fluxes[-1]), sorted_path_fluxes[-1])

Path index 	 Integrated Flux 	 Rel. Flux (%) 		 Path steps

Path #0: 	 7.530E-09 		 0.15 		 ['ethane(1)', 'C2H5(3)', 'H(4)', 'C2H4(8)', 'C2H3(13)', 'C#C(25)']
Path #1: 	 5.869E-07 		 11.58 		 ['ethane(1)', 'C2H5(3)', 'C2H4(8)', 'C2H3(13)', 'C#C(25)']

	 Most Significant Path 
Path index 	 Integrated Flux 	 Rel. Flux (%)
Path #1: 	 5.869E-07 		 11.58


### [user input] output2: visualize the path of interest

Please specify

- the path index you want to visualize

In [49]:
path_index_investigate = 0
path = paths[path_index_investigate]
path_steps = len(path) - 1
print ''
print '\t Pathway Report '
print '======================================'
print 'The pathway you are intested in has {0} steps.'.format(len(path))
for step in range(path_steps):
    step_pair = (path[step], path[step+1])
    h_abs_rxns = []
    disp_rxns = []
    Pdep_rxns = []
    
    print ""
    if showIntegratedFlux:
        print "Step{0} \t\t\t\t Integrated Flux \t Rel. Flux (%)".format(step+1)
        print '***********************************************************************'
        print "{1} --> {2} \t\t {3:.3E} \t\t {4:.2f}".\
        format(step+1, step_pair[0], step_pair[1], G[step_pair[0]][step_pair[1]]['total_flux'], 
               G[step_pair[0]][step_pair[1]]['total_flux']/total_reactant_consumption*100)
        print '***********************************************************************'
    else:
        print "Step{0} \t\t\t\t Flux".format(step+1)
        print '******************************************'
        print "{1} --> {2} \t\t {3:.3E}".\
        format(step+1, step_pair[0], step_pair[1], G[step_pair[0]][step_pair[1]]['total_flux'])
        print '******************************************'
    
    if step_pair not in graph_edges_dict_simple:
        step_pair = (step_pair[1], step_pair[0])
    
    print ''
    for rxn in graph_edges_dict_simple[step_pair]:
        if isinstance(rxn, PDepReaction):
            Pdep_rxns.append(rxn)
        elif rxn.family == "H_Abstraction":
            h_abs_rxns.append(rxn)
        elif rxn.family == "Disproportionation":
            disp_rxns.append(rxn)
        else:
            print "Example reaction: rxn#{0}".format(rxn.index)
            print ''
            print str(rxn)
            display(rxn)
            print 'Forward rate coefficient at {0} K: {1:.2E}'.format(T, rxn.getRateCoefficient(T))
            reverseRate = rxn.generateReverseRateCoefficient()
            print 'Reverse rate coefficient at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))
            print ''
    if len(h_abs_rxns) > 0: 
        
        print "Example H_Abstraction: rxn#{0} (1/{1} H_Abs): ".format(h_abs_rxns[0].index, len(h_abs_rxns)) 
        print ''
        print str(h_abs_rxns[0])
        display(h_abs_rxns[0])
        print 'Forward rate coefficient at {0} K: {1:.2E}'.format(T, h_abs_rxns[0].getRateCoefficient(T))
        reverseRate = h_abs_rxns[0].generateReverseRateCoefficient()
        print 'Reverse rate coefficient at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))
        print ''
    if len(disp_rxns) > 0: 
        
        print "Example Disproportionation: rxn#{0} (1/{1} Disp): ".format(disp_rxns[0].index, len(disp_rxns)) 
        print ''
        print str(disp_rxns[0])
        display(disp_rxns[0])
        print 'Forward rate coefficient at {0} K: {1:.2E}'.format(T, disp_rxns[0].getRateCoefficient(T))
        reverseRate = disp_rxns[0].generateReverseRateCoefficient()
        print 'Reverse rate coefficient at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))
        print ''
    if len(Pdep_rxns) > 0: 
        
        print "Example Pressure-dependent Rxn: rxn#{0} (1/{1} Pdep_rxns): ".format(Pdep_rxns[0].index, len(Pdep_rxns)) 
        print ''
        print str(Pdep_rxns[0])
        display(Pdep_rxns[0])
        print 'Forward rate coefficient at {0} K and {1} Pa: {2:.2E}'.format(T, P, Pdep_rxns[0].getRateCoefficient(T, P))
        reverseRate = Pdep_rxns[0].generateReverseRateCoefficient()
        print 'Reverse rate coefficient at {0} K and {1} Pa: {2:.2E}'.format(T, P, reverseRate.getRateCoefficient(T, P))
        print ''


	 Pathway Report 
The pathway you are intested in has 6 steps.

Step1 				 Integrated Flux 	 Rel. Flux (%)
***********************************************************************
ethane(1) --> C2H5(3) 		 4.623E-06 		 91.22
***********************************************************************

Example reaction: rxn#3

C[CH2](3) + [H](4) <=> ethane(1)


Forward rate coefficient at 1350 K: 1.00E+08
Reverse rate coefficient at 1350 K: 1.28E+00

Example H_Abstraction: rxn#2 (1/3 H_Abs): 

[CH3](2) + ethane(1) <=> C[CH2](3) + C(6)


Forward rate coefficient at 1350 K: 1.84E+04
Reverse rate coefficient at 1350 K: 1.57E+03

Example Disproportionation: rxn#20 (1/2 Disp): 

[CH]=C(13) + C[CH2](3) <=> C#C(25) + ethane(1)


Forward rate coefficient at 1350 K: 2.46E+06
Reverse rate coefficient at 1350 K: 2.29E-02


Step2 				 Integrated Flux 	 Rel. Flux (%)
***********************************************************************
C2H5(3) --> H(4) 		 4.509E-06 		 88.98
***********************************************************************

Example reaction: rxn#5

C=C(8) + [H](4) <=> C[CH2](3)


Forward rate coefficient at 1350 K: 4.31E+07
Reverse rate coefficient at 1350 K: 3.56E+07


Step3 				 Integrated Flux 	 Rel. Flux (%)
***********************************************************************
H(4) --> C2H4(8) 		 7.530E-09 		 0.15
***********************************************************************

Example reaction: rxn#13

[CH]=C(13) + [H](4) <=> C=C(8)


...

Forward rate coefficient at 1350 K: 1.21E+08
Reverse rate coefficient at 1350 K: 2.89E-02

Example H_Abstraction: rxn#14 (1/3 H_Abs): 

[CH]=C(13) + C(6) <=> C=C(8) + [CH3](2)


Forward rate coefficient at 1350 K: 1.13E+05
Reverse rate coefficient at 1350 K: 2.47E+04

Example Disproportionation: rxn#17 (1/2 Disp): 

[CH]=C(13) + C[CH2](3) <=> C=C(8) + C=C(8)


Forward rate coefficient at 1350 K: 2.94E+06
Reverse rate coefficient at 1350 K: 8.50E-04


Step5 				 Integrated Flux 	 Rel. Flux (%)
***********************************************************************
C2H3(13) --> C#C(25) 		 5.870E-07 		 11.58
***********************************************************************

Example reaction: rxn#18

C#C(25) + [H](4) <=> [CH]=C(13)


Forward rate coefficient at 1350 K: 6.38E+07
Reverse rate coefficient at 1350 K: 8.76E+07

Example Disproportionation: rxn#19 (1/4 Disp): 

[CH]=C(13) + [CH3](2) <=> C#C(25) + C(6)


Forward rate coefficient at 1350 K: 2.46E+06
Reverse rate coefficient at 1350 K: 1.96E-03



## [user input] Functionality 3: find paths from a particular species

Please specify

- source species

- depth of pathway you want search, e.g., depth = 2 means search pathways 2 steps down from source species

- path_top_list, e.g., [0, 3] means 1st step of the pathway takes the most significant branch and 2nd step would be the third most siginificant branch

In [50]:
source = "ethane(1)"
depth = 2
path_top_list = [0, 0]

In [51]:
current_node = source

print ''
print '\t Pathway Report '
print '======================================'
print 'The pathway you are intested in has {0} steps.'.format(depth)
for step in range(depth):
    print "\n"    
    nextNode_flux_list = [(next_node, G[current_node][next_node]['total_flux']) for next_node in G[current_node]]
    sorted_nextNode_flux_list = sorted(nextNode_flux_list, key=lambda tup: -tup[1])
    
    # choose the top one as next node
    tup = sorted_nextNode_flux_list[path_top_list[step]]
    next_node = tup[0]
    step_flux = tup[1]
    
    print ""
    if showIntegratedFlux:
        print "Step{0} \t\t\t\t Integrated Flux \t Rel. Flux (%)".format(step+1)
        print '************************************************************************'
        print "{0} --> {1} \t\t {2:.3E} \t\t {3:.2f}".\
        format(current_node, next_node, step_flux, step_flux/total_reactant_consumption*100)
        print '************************************************************************'
    else:
        print "Step{0} \t\t\t\t Flux".format(step+1)
        print '******************************************'
        print "{0} --> {1} \t\t {2:.3E}".\
        format(current_node, next_node, step_flux)
        print '******************************************'

    
    step_pair = (current_node, next_node)
    if step_pair not in graph_edges_dict_simple:
        step_pair = (next_node, current_node)
    
    h_abs_rxns = []
    disp_rxns = []
    Pdep_rxns = []
    for rxn in graph_edges_dict_simple[step_pair]:
        if isinstance(rxn, PDepReaction):
            Pdep_rxns.append(rxn)
        elif rxn.family == "H_Abstraction":
            h_abs_rxns.append(rxn)
        elif rxn.family == "Disproportionation":
            disp_rxns.append(rxn)
        else:
            print ''
            print "Example reaction: rxn#{0}".format(rxn.index)
            print ''
            if showIntegratedFlux:
            #integrate the flux using a trapezoidal rule
                integrated_flux = 0
                for timepoint in range(timepoint_index-1):
                    integrated_flux += 0.5 * (graph_edges_dict_simple[step_pair][rxn][timepoint+1] 
                                              + graph_edges_dict_simple[step_pair][rxn][timepoint]) \
                    * (firstColDict['Time_(sec)'][timepoint+1] - firstColDict['Time_(sec)'][timepoint])
                print "{0}: integrated flux = {1:.3E} ".format(str(rxn), integrated_flux)
            else:
                print "{0}: flux = {1:.3E} ".format(str(rxn), graph_edges_dict_simple[step_pair][rxn][timepoint_index])
            display(rxn)
            print 'Forward rate coefficient at {0} K: {1:.2E}'.format(T, rxn.getRateCoefficient(T))
            reverseRate = rxn.generateReverseRateCoefficient()
            print 'Reverse rate coefficient at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))
            print ''
    if len(h_abs_rxns) > 0:
        print ''
        print "Example H_Abstraction: rxn#{0}(1/{1} H_Abs)".format(h_abs_rxns[0].index, len(h_abs_rxns))
        print ''
        print str(h_abs_rxns[0])
        display(h_abs_rxns[0])
        print 'Forward rate coefficient at {0} K: {1:.2E}'.format(T, h_abs_rxns[0].getRateCoefficient(T))
        reverseRate = h_abs_rxns[0].generateReverseRateCoefficient()
        print 'Reverse rate coefficient at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))
        print ''
    if len(disp_rxns) > 0: 
        print ''
        print "Example Disproportionation: rxn#{0}(1/{1} Disp)".format(disp_rxns[0].index, len(disp_rxns))
        print ''
        print str(disp_rxns[0])
        display(disp_rxns[0])
        print 'Forward rate coefficient at {0} K: {1:.2E}'.format(T, disp_rxns[0].getRateCoefficient(T))
        reverseRate = disp_rxns[0].generateReverseRateCoefficient()
        print 'Reverse rate coefficient at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))
        print ''
    if len(Pdep_rxns) > 0: 
        print ''
        print "Example Pressure-dependent Rxn: rxn#{0} (1/{1} Pdep_rxns): ".format(Pdep_rxns[0].index, len(Pdep_rxns)) 
        print ''
        print str(Pdep_rxns[0])
        display(Pdep_rxns[0])
        print 'Forward rate coefficient at {0} K and {1} Pa: {2:.2E}'.format(T, P, Pdep_rxns[0].getRateCoefficient(T, P))
        reverseRate = Pdep_rxns[0].generateReverseRateCoefficient()
        print 'Reverse rate coefficient at {0} K and {1} Pa: {2:.2E}'.format(T, P, reverseRate.getRateCoefficient(T, P))
        print ''
    
    current_node = next_node


	 Pathway Report 
The pathway you are intested in has 2 steps.



Step1 				 Integrated Flux 	 Rel. Flux (%)
************************************************************************
ethane(1) --> C2H5(3) 		 4.623E-06 		 91.22
************************************************************************

Example reaction: rxn#3

C[CH2](3) + [H](4) <=> ethane(1): integrated flux = 1.200E-07 


Forward rate coefficient at 1350 K: 1.00E+08
Reverse rate coefficient at 1350 K: 1.28E+00


Example H_Abstraction: rxn#2(1/3 H_Abs)

[CH3](2) + ethane(1) <=> C[CH2](3) + C(6)


Forward rate coefficient at 1350 K: 1.84E+04
Reverse rate coefficient at 1350 K: 1.57E+03


Example Disproportionation: rxn#20(1/2 Disp)

[CH]=C(13) + C[CH2](3) <=> C#C(25) + ethane(1)


Forward rate coefficient at 1350 K: 2.46E+06
Reverse rate coefficient at 1350 K: 2.29E-02




Step2 				 Integrated Flux 	 Rel. Flux (%)
************************************************************************
C2H5(3) --> C2H4(8) 		 4.622E-06 		 91.21
************************************************************************

Example reaction: rxn#5

C=C(8) + [H](4) <=> C[CH2](3): integrated flux = 4.509E-06 


Forward rate coefficient at 1350 K: 4.31E+07
Reverse rate coefficient at 1350 K: 3.56E+07


Example Disproportionation: rxn#6(1/4 Disp)

[CH3](2) + C[CH2](3) <=> C=C(8) + C(6)


Forward rate coefficient at 1350 K: 4.89E+06
Reverse rate coefficient at 1350 K: 6.46E-03



### More detail for the H_Abstration and Disproportionation in last step of the pathway

In [52]:
print ''
print 'The step you see currently is between the two species {0}'.format(step_pair)
print ''
print 'Positive flux in below means positive production of 1st node of the pair.'
print ''
print 'This step have {0} H_Abstration and Disproportionation in total.'.format(len(h_abs_rxns + disp_rxns))

flux_threshold = 1e-9

print ''
print 'TO avoid too much printout, the reactions shown below have fluxes above {0}.'.format(flux_threshold)

total_flux = 0
rxn_flux_tups = []
for rxn in h_abs_rxns + disp_rxns:
    if showIntegratedFlux:
    #integrate the flux using a trapezoidal rule
        integrated_flux = 0
        for timepoint in range(timepoint_index-1):
                integrated_flux += 0.5 * (graph_edges_dict_simple[step_pair][rxn][timepoint+1] \
                    + graph_edges_dict_simple[step_pair][rxn][timepoint]) \
                    * (firstColDict['Time_(sec)'][timepoint+1] - firstColDict['Time_(sec)'][timepoint])
        rxn_flux_tups.append((rxn, integrated_flux))
    else:
        flux = graph_edges_dict_simple[step_pair][rxn][timepoint_index]
        rxn_flux_tups.append((rxn, flux))
    

rxn_flux_tups = sorted(rxn_flux_tups, key=lambda tup: tup[1], reverse=False)
for tup in rxn_flux_tups:
    rxn = tup[0]
    flux = tup[1]
    if flux > flux_threshold:
        total_flux += flux
        print ''
        print "**********************************************************************************"
        if showIntegratedFlux:
            print "rxn#{0}: {1}: integrated flux = {2:.3E} ".format(rxn.index, str(rxn), flux) 
        else:
            print "rxn#{0}: {1}: flux = {2:.3E} ".format(rxn.index, str(rxn), flux)
        display(rxn) 
print "***************************************"
print ''
if showIntegratedFlux:
    print "TOTAL integrated flux from h_abs and disp is {0:.3E}.".format(total_flux)
else:
    print "TOTAL flux from h_abs and disp is {0:.3E}.".format(total_flux)



The step you see currently is between the two species ('C2H4(8)', 'C2H5(3)')

Positive flux in below means positive production of 1st node of the pair.

This step have 4 H_Abstration and Disproportionation in total.

TO avoid too much printout, the reactions shown below have fluxes above 1e-09.

**********************************************************************************
rxn#17: [CH]=C(13) + C[CH2](3) <=> C=C(8) + C=C(8): integrated flux = 3.171E-09 



**********************************************************************************
rxn#9: C[CH2](3) + [H](4) <=> [H][H](12) + C=C(8): integrated flux = 2.655E-08 



**********************************************************************************
rxn#6: [CH3](2) + C[CH2](3) <=> C=C(8) + C(6): integrated flux = 3.425E-08 



**********************************************************************************
rxn#7: C[CH2](3) + C[CH2](3) <=> C=C(8) + ethane(1): integrated flux = 4.895E-08 


***************************************

TOTAL integrated flux from h_abs and disp is 1.129E-07.


## [user input] Functionality 4: find paths towards a particular species

Similar to functionality 2, please specify

Please specify

- target species

- depth of pathway you want search, e.g., depth = 2 means search pathways 2 steps up beyond target species

- path_top_list, e.g., [0, 3] means 1st step of the pathway takes the most significant branch and 2nd step would be the third most siginificant branch

In [53]:
target = "C#C(25)"
depth = 2
path_top_list = [0, 0]

In [54]:
current_node = target
print ''
print '\t Pathway Report '
print '======================================'
print 'The pathway you are intested in has {0} steps.'.format(depth)
print ''
print "total product flux for {0} is {1:.3E}.".format(target, spc_total_rop_dict[target][1][timepoint_index])
for step in range(depth):
    print "\n"
    prev_nodes = []
    for node1 in G:
        if current_node in G[node1]:
            prev_nodes.append(node1)
    prevNode_flux_list = [(prev_node, G[prev_node][current_node]['total_flux']) for prev_node in prev_nodes]
    sorted_prevNode_flux_list = sorted(prevNode_flux_list, key=lambda tup: -tup[1])
    
    # choose the top one as next node
    tup = sorted_prevNode_flux_list[path_top_list[step]]
    prev_node = tup[0]
    step_flux = tup[1]
    
    
    
    print ""
    if showIntegratedFlux:
        print "Step{0} \t\t\t\t Integrated Flux \t Rel. Flux (%)".format(step+1)
        print '************************************************************************'
        print "{0} --> {1} \t\t {2:.3E} \t\t {3:.2f}".\
        format(current_node, next_node, step_flux, step_flux/total_reactant_consumption*100)
        print '************************************************************************'
    else:
        print "Step{0} \t\t\t\t Flux".format(step+1)
        print '******************************************'
        print "{0} --> {1} \t\t {2:.3E}".\
        format(current_node, next_node, step_flux)
        print '******************************************'
    
    step_pair = (prev_node, current_node)
    if step_pair not in graph_edges_dict_simple:
        step_pair = (current_node, prev_node)
    
    h_abs_rxns = []
    disp_rxns = []
    Pdep_rxns = []
    for rxn in graph_edges_dict_simple[step_pair]:
        if isinstance(rxn, PDepReaction):
            Pdep_rxns.append(rxn)
        elif rxn.family == "H_Abstraction":
            h_abs_rxns.append(rxn)
        elif rxn.family == "Disproportionation":
            disp_rxns.append(rxn)
        else:
            print ''
            print "Example reaction: rxn#{0}".format(rxn.index)
            print ''
            if showIntegratedFlux:
            #integrate the flux using a trapezoidal rule
                integrated_flux = 0
                for timepoint in range(timepoint_index-1):
                    integrated_flux += 0.5 * (graph_edges_dict_simple[step_pair][rxn][timepoint+1] 
                                              + graph_edges_dict_simple[step_pair][rxn][timepoint]) \
                    * (firstColDict['Time_(sec)'][timepoint+1] - firstColDict['Time_(sec)'][timepoint])
                print "{0}: integrated flux = {1:.3E} ".format(str(rxn), integrated_flux)
            else:
                print "{0}: flux = {1:.3E} ".format(str(rxn), graph_edges_dict_simple[step_pair][rxn][timepoint_index])
            display(rxn)
            print 'Forward rate coefficient at {0} K: {1:.2E}'.format(T, rxn.getRateCoefficient(T))
            reverseRate = rxn.generateReverseRateCoefficient()
            print 'Reverse rate coefficient at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))
            
    if len(h_abs_rxns) > 0:
        print ''
        print "Example H_Abstraction: rxn#{0}(1/{1} H_Abs)".format(h_abs_rxns[0].index, len(h_abs_rxns))
        print ''
        print str(h_abs_rxns[0])
        display(h_abs_rxns[0])
        print 'Forward rate coefficient at {0} K: {1:.2E}'.format(T, h_abs_rxns[0].getRateCoefficient(T))
        reverseRate = h_abs_rxns[0].generateReverseRateCoefficient()
        print 'Reverse rate coefficient at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))
        print ''
    if len(disp_rxns) > 0: 
        print ''
        print "Example Disproportionation: rxn#{0}(1/{1} Disp)".format(disp_rxns[0].index, len(disp_rxns))
        print ''
        print str(disp_rxns[0])
        display(disp_rxns[0])
        print 'Forward rate coefficient at {0} K: {1:.2E}'.format(T, disp_rxns[0].getRateCoefficient(T))
        reverseRate = disp_rxns[0].generateReverseRateCoefficient()
        print 'Reverse rate coefficient at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))
        print ''
    if len(Pdep_rxns) > 0: 
        print ''
        print "Example Pressure-dependent Rxn: rxn#{0} (1/{1} Pdep_rxns): ".format(Pdep_rxns[0].index, len(Pdep_rxns)) 
        print ''
        print str(Pdep_rxns[0])
        display(Pdep_rxns[0])
        print 'Forward rate coefficient at {0} K and {1} Pa: {2:.2E}'.format(T, P, Pdep_rxns[0].getRateCoefficient(T, P))
        reverseRate = Pdep_rxns[0].generateReverseRateCoefficient()
        print 'Reverse rate coefficient at {0} K and {1} Pa: {2:.2E}'.format(T, P, reverseRate.getRateCoefficient(T, P))
        print ''
    
    current_node = prev_node


	 Pathway Report 
The pathway you are intested in has 2 steps.

total product flux for C#C(25) is 1.685E-04.



Step1 				 Integrated Flux 	 Rel. Flux (%)
************************************************************************
C#C(25) --> C2H4(8) 		 5.870E-07 		 11.58
************************************************************************

Example reaction: rxn#18

C#C(25) + [H](4) <=> [CH]=C(13): integrated flux = 5.793E-07 


Forward rate coefficient at 1350 K: 6.38E+07
Reverse rate coefficient at 1350 K: 8.76E+07

Example Disproportionation: rxn#19(1/4 Disp)

[CH]=C(13) + [CH3](2) <=> C#C(25) + C(6)


Forward rate coefficient at 1350 K: 2.46E+06
Reverse rate coefficient at 1350 K: 1.96E-03




Step2 				 Integrated Flux 	 Rel. Flux (%)
************************************************************************
C2H3(13) --> C2H4(8) 		 5.869E-07 		 11.58
************************************************************************

Example reaction: rxn#13

[CH]=C(13) + [H](4) <=> C=C(8): integrated flux = -7.530E-09 


Forward rate coefficient at 1350 K: 1.21E+08
Reverse rate coefficient at 1350 K: 2.89E-02

Example H_Abstraction: rxn#14(1/3 H_Abs)

[CH]=C(13) + C(6) <=> C=C(8) + [CH3](2)


Forward rate coefficient at 1350 K: 1.13E+05
Reverse rate coefficient at 1350 K: 2.47E+04


Example Disproportionation: rxn#17(1/2 Disp)

[CH]=C(13) + C[CH2](3) <=> C=C(8) + C=C(8)


Forward rate coefficient at 1350 K: 2.94E+06
Reverse rate coefficient at 1350 K: 8.50E-04



### More detail for the H_Abstration and Disproportionation in last step of the pathway

In [55]:
print ''
print 'The step you see currently is between the two species {0}'.format(step_pair)
print ''
print 'Positive flux in below means positive production of 1st node of the pair.'
print ''
print 'This step have {0} H_Abstration and Disproportionation in total.'.format(len(h_abs_rxns + disp_rxns))

flux_threshold = 1e-9

print ''
print 'TO avoid too much printout, the reactions shown below have fluxes above {0}.'.format(flux_threshold)

total_flux = 0
rxn_flux_tups = []
for rxn in h_abs_rxns + disp_rxns:
    if showIntegratedFlux:
    #integrate the flux using a trapezoidal rule
        integrated_flux = 0
        for timepoint in range(timepoint_index-1):
                integrated_flux += 0.5 * (graph_edges_dict_simple[step_pair][rxn][timepoint+1] \
                    + graph_edges_dict_simple[step_pair][rxn][timepoint]) \
                    * (firstColDict['Time_(sec)'][timepoint+1] - firstColDict['Time_(sec)'][timepoint])
        rxn_flux_tups.append((rxn, integrated_flux))
    else:
        flux = graph_edges_dict_simple[step_pair][rxn][timepoint_index]
        rxn_flux_tups.append((rxn, flux))

rxn_flux_tups = sorted(rxn_flux_tups, key=lambda tup: tup[1], reverse=False)
for tup in rxn_flux_tups:
    rxn = tup[0]
    flux = tup[1]
    if flux > flux_threshold:
        total_flux += flux
        print ''
        print "**********************************************************************************"
        if showIntegratedFlux:
            print "rxn#{0}: {1}: integrated flux = {2:.3E} ".format(rxn.index, str(rxn), flux) 
        else:
            print "rxn#{0}: {1}: flux = {2:.3E} ".format(rxn.index, str(rxn), flux)
        display(rxn) 
print "***************************************"
print ''
if showIntegratedFlux:
    print "TOTAL integrated flux from h_abs and disp is {0:.3E}.".format(total_flux)
else:
    print "TOTAL flux from h_abs and disp is {0:.3E}.".format(total_flux)


The step you see currently is between the two species ('C2H3(13)', 'C2H4(8)')

Positive flux in below means positive production of 1st node of the pair.

This step have 5 H_Abstration and Disproportionation in total.

TO avoid too much printout, the reactions shown below have fluxes above 1e-09.

**********************************************************************************
rxn#14: [CH]=C(13) + C(6) <=> C=C(8) + [CH3](2): integrated flux = 1.459E-07 



**********************************************************************************
rxn#16: [CH]=C(13) + [H][H](12) <=> C=C(8) + [H](4): integrated flux = 5.001E-07 


***************************************

TOTAL integrated flux from h_abs and disp is 6.461E-07.
